In [25]:

import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.decomposition import DictionaryLearning


np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# ------------------------ Load Data ------------------------
measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# ------------------------ Feature Extraction ------------------------
# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:100, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:100, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude)) # (100, 753) 251*3

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude)) # (100, 753)

# ------------------------ Dictionary ------------------------
# labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0]) # 100  # Label '0' for Alice.
eve_labels = np.ones(eve_features.shape[0]) # 100       # Label '1' for Eve.

atoms = np.vstack((alice_features, eve_features))  # (200, 753)
true_labels = np.hstack((alice_labels, eve_labels)) # (200,)

# Dictionary
# dictionary = atoms #(200, 753)

dl = DictionaryLearning(10, transform_algorithm='lasso_lars')
dictionary = dl.fit_transform(atoms.T) # (753, 10)

In [31]:
# ------------------------ Test ------------------------
test_real = eve_CIRs[0, :, 0]
test_imag = eve_CIRs[0, :, 1]
test_magnitude = np.abs(test_real + 1j * test_imag)
test_signal = np.hstack((test_real, test_imag, test_magnitude))


# y = (200, 753) X (753,)
y = dictionary.T @ test_signal
# print(dictionary.shape)
# y (200, )



# ------------------------ Coefficient ------------------------

# OMP object
omp = OrthogonalMatchingPursuit()

omp.fit(dictionary.T, y)

# coefficients
coefficients = omp.coef_

print("Coefficients:", coefficients)


Coefficients: [      0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
       0.               0.               0.               0.
 -108623.78758766       0.         -125496.95499806       0.
  -63624.11347251       0.         -122046.81655791       0.
       0.               0.         -134892.20907313       0.
       0. 

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
